# 06_과적합_일반화_그리드서치_파이프라인 TODO

# TODO Adult dataset

- 전처리
    - 범주형
        - 결측치는 최빈값으로 대체한다.
        - 원핫인코딩 처리한다.
    - 연속형
        - 결측치는 중앙값으로 대체한다.
        - StandardScaling을 한다.
- Model: `sklearn.linear_model.LogisticRegression(max_iter=2000)` 를 사용
- Pipeline을 이용해 전처리와 모델을 묶어준다.

## 데이터 로드

In [1]:
import pandas as pd
cols = ['age', 'workclass','fnlwgt','education', 'education-num', 'marital-status', 'occupation','relationship', 'race', 'gender','capital-gain','capital-loss', 'hours-per-week','native-country', 'income']
data = pd.read_csv(
    'C:/SKN13SM/SKN13_Sumin/08_머신러닝/data/adult.data', 
    header=None, 
    names=cols,
    na_values='?', 
    skipinitialspace=True 
)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


- header = None => 인덱스가 0, 1, ...로 됨 => 인덱스를 cols 리스트를 이용해 지정해줌

In [2]:
categorical_columns = ['workclass','education','marital-status', 'occupation','relationship','race','gender','native-country']
numeric_columns = ['age','fnlwgt', 'education-num','capital-gain','capital-loss','hours-per-week']
target = "income"  # 14번째 컬럼

categorical_columns_index = [1, 3, 5, 6, 7, 8, 9, 13]
numeric_columns_index = [0, 2, 4, 10, 11, 12]

ColumnTransformer에서 열을 줄 때, 열의 이름(feature 이름)이나 인덱스 번호를 줄 수 있다. 

df을 입력으로 넣을 경우는 이름과 번호 둘 다 가능하지만, ndarray는 이름이 없음 => 인덱스를 넣어야 한다.

따라서, 인덱스 번호를 넣는 게 좀 더 안전하다~~~

In [5]:
y

0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
32556    <=50K
32557     >50K
32558    <=50K
32559    <=50K
32560     >50K
Name: income, Length: 32561, dtype: object

In [3]:
#데이터셋 분리
from sklearn.model_selection import train_test_split

X, y = data.drop(columns=['income']), data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

In [5]:
X_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
28291,25,Private,193379,Assoc-acdm,12,Never-married,Craft-repair,Not-in-family,White,Male,0,0,45,United-States
28636,55,Federal-gov,176904,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States
7919,30,Private,284395,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,50,United-States
24861,17,Private,239346,10th,6,Never-married,Other-service,Own-child,White,Male,0,0,18,United-States
23480,51,Private,57698,HS-grad,9,Married-spouse-absent,Other-service,Unmarried,White,Female,0,0,40,United-States


In [6]:
y_train.head()

28291    <=50K
28636    <=50K
7919     <=50K
24861    <=50K
23480    <=50K
Name: income, dtype: object

## 파이프라인 생성

In [7]:
# 파이프라인 생성

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer   # 결측치값 대체.
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

In [8]:
########### 컬럼별 전처리 프로세스를 pipeline으로 묶기.
### 수치형 컬럼들에 적용할 전처리 프로세스.
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")), # 1. 결측치 처리
    ("scaler", StandardScaler())   # 2. Feature Scaling
])
### 범주형 컬럼들에 적용할 전처리 프로세스
cate_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")), 
    ("ohe", OneHotEncoder(handle_unknown='ignore'))   # handle_unknown='ignore' - 학습할 때 없었던 class는 0으로 처리.
])

preprocessor = ColumnTransformer([
    ("category", cate_pipeline, categorical_columns_index), 
    ("number", num_pipeline, numeric_columns_index)
])

In [9]:
import sklearn

In [10]:
from sklearn.linear_model import LogisticRegression
steps = [
    ("preprocessor", preprocessor),
    ("LogisticRegression", sklearn.linear_model.LogisticRegression(max_iter=2000))
]

pipeline = Pipeline(steps, verbose=True)

In [11]:
pipeline.fit(X_train, y_train)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.1s
[Pipeline]  (step 2 of 2) Processing LogisticRegression, total=   0.2s


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('category',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [1, 3, 5, 6, 7, 8, 9, 13]),
                                                 ('number',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  [0, 2, 4, 10, 11, 12])])),
                ('LogisticRegression', LogisticRegression(max_iter=2000))],
         verbose=True)

In [43]:
pred_train = pipeline.predict(X_train)

In [44]:
pred_train

array(['<=50K', '>50K', '<=50K', ..., '<=50K', '>50K', '<=50K'],
      shape=(24420,), dtype=object)

## 결과 확인

In [30]:
pred_test = pipeline.predict(X_test)

In [31]:
y_train

28291    <=50K
28636    <=50K
7919     <=50K
24861    <=50K
23480    <=50K
         ...  
28261     >50K
15326    <=50K
284      <=50K
13784    <=50K
3166      >50K
Name: income, Length: 24420, dtype: object

In [47]:
pred_train # y_train처럼 시리즈로 안 바꿔도 됨

array(['<=50K', '>50K', '<=50K', ..., '<=50K', '>50K', '<=50K'],
      shape=(24420,), dtype=object)

In [48]:
# from metrics import print_binary_classification_metrics

# print_binary_classification_metrics(y_train, pred_train, title="Train set 정확도")

# ValueError: pos_label=1 is not a valid label. It should be one of ['<=50K', '>50K']
# 0, 1이 아닌, ['<=50K', '>50K']으로 되어있어서

In [49]:
# 문자열 라벨을 숫자로 변환
y_train_binary = (y_train == '>50K').astype(int)
pred_train_binary = (pred_train == '>50K').astype(int)

# 그리고 이렇게 사용
print_binary_classification_metrics(y_train_binary, pred_train_binary, title="Train set 정확도")


Train set 정확도
정확도: 0.8527436527436527
재현율: 0.6043189933684747
정밀도: 0.7368857557536803
F1 점수: 0.664050822122571


In [50]:
pred_test = pipeline.predict(X_test)

In [51]:
# 문자열 라벨을 숫자로 변환
y_test_binary = (y_test == '>50K').astype(int)
pred_test_binary = (pred_test == '>50K').astype(int)

# 그리고 이렇게 사용
print_binary_classification_metrics(y_test_binary, pred_test_binary, title="Train set 정확도")


Train set 정확도
정확도: 0.8481758997666135
재현율: 0.588265306122449
정밀도: 0.7288242730720607
F1 점수: 0.6510446075663467
